In [3]:
import optuna

In [4]:
from stesml.model_tools import build_train_test_model

In [9]:
data_dir = "../data/Sulfur_Models/"
model_type = 'NN' # Options: NN, XGBoost, RandomForest
target = 'Tavg' # Options: Tavg, h
metric = 'rmse' # Options: rmse, r2
n_shuffles = 5

if metric == 'rmse':
    direction = 'minimize'
elif metric == 'r2':
    directon = 'maximize'

In [10]:
def objective(trial):   
    if model_type == 'NN':
        scale = True
        n_layers = trial.suggest_int("n_layers", 1, 5)
        n_hidden_units = trial.suggest_int("n_hidden_units", 10, 100)
        batch_size = trial.suggest_int("batch_size", 10, 10000, log=True)
        epochs = trial.suggest_int("epochs", 1, 20)
        result = build_train_test_model(data_dir, model_type, target, metric, scale, n_layers, n_hidden_units, batch_size, epochs, n_shuffles=n_shuffles)
    elif model_type == 'XGBoost':
        scale = False
        n_estimators = trial.suggest_int("n_estimators", 50, 10000, log=True)
        result = build_train_test_model(data_dir, model_type, target, metric, scale, n_estimators=n_estimators, n_shuffles=n_shuffles)
    elif model_type == 'RandomForest':
        scale = False
        n_estimators = trial.suggest_int("n_estimators", 1, 100, log=True)
        result = build_train_test_model(data_dir, model_type, target, metric, scale, n_estimators=n_estimators, n_shuffles=n_shuffles)
    
    return result

In [11]:
study = optuna.create_study(direction=direction)

[I 2022-06-29 16:37:51,655] A new study created in memory with name: no-name-2e40b9dd-4df3-469d-8b7d-7dd3dfcfa316


In [12]:
study.optimize(objective, n_trials=50)

1322/1322 [==============================] - 3s 2ms/step - loss: 0.0434 - val_loss: 0.0028


[W 2022-06-29 16:38:14,146] Trial 0 failed because of the following error: ParserError("Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.")
Traceback (most recent call last):
  File "/Users/kmenear/Projects/envSulfurTES/lib/python3.9/site-packages/optuna/study/_optimize.py", line 213, in _run_trial
    value_or_values = func(trial)
  File "/var/folders/cy/yhfymg2j61gfl448mb_195qxlvt1vt/T/ipykernel_56092/3298196419.py", line 8, in objective
    result = build_train_test_model(data_dir, model_type, target, metric, scale, n_layers, n_hidden_units, batch_size, epochs, n_shuffles=n_shuffles)
  File "/Users/kmenear/Projects/envSulfurTES/lib/python3.9/site-packages/stesml-0+unknown-py3.9.egg/stesml/model_tools.py", line 117, in build_train_test_model
    X_train, y_train, X_test, y_test, scaler_x, scaler_y = get_train_and_test_data(scenario_index, train_index, test_index, target, scale)
  File "/Users/kmenear/Projects/envSulfurTES/lib/python3.9/site-p

Result: 3.0518
RMSE AVG: 3.0518


ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [17]:
best_params = study.best_params

In [18]:
best_params

{'n_layers': 2, 'n_hidden_units': 79, 'batch_size': 339, 'epochs': 7}

In [21]:
best_value = study.best_value

In [22]:
best_value

1.0137617092656621